## temel bilesen regresyonu - pcr - tahmin


In [1]:
import pandas as pd

# hitters.csv = basketbol oyuncularinin verilerini bulunduran veri seti

hit = pd.read_csv("Hitters.csv")
df = hit.copy()

df = df.dropna()
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [2]:
y = df["Salary"]

In [3]:
x_ = df.drop(
    ["Salary", "League", "Division", "NewLeague"],
    axis=1,
).astype("float")
x_.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0


In [4]:
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])
dms.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,False,True,False,True,False,True
2,True,False,False,True,True,False
3,False,True,True,False,False,True
4,False,True,True,False,False,True
5,True,False,False,True,True,False


In [5]:
x = pd.concat([x_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis=1)
x.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,True,True,True
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,False,True,False
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,True,False,True
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,True,False,True
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,False,True,False


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42
)

training = df.copy()

In [7]:
from sklearn.decomposition import PCA

pca = PCA()

In [8]:
from sklearn.preprocessing import scale

x_reduced_train = pca.fit_transform(scale(x_train))

In [9]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

In [10]:
pcr_model = lm.fit(x_reduced_train, y_train)

#### ========================================================

#### **pcr - tahmin**


In [11]:
y_pred = pcr_model.predict(x_reduced_train)
y_pred[0:5]

# tahmin edilen degerler

array([377.44484744, 802.19452124, 495.60987745, 112.53177731,
       426.21613066])

In [12]:
import numpy as np
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_train, y_pred))

# train setimiz icin hata hesaplama

289.3292825564976

In [13]:
df["Salary"].mean()

535.9258821292775

In [14]:
from sklearn.metrics import r2_score

r2_score(y_train, y_pred)

# r2 degerleri

0.5770075250410179

In [15]:
pca2 = PCA()

# test hatamizi inceliyoruz

In [16]:
x_reduced_test = pca2.fit_transform(scale(x_test))

# indirgeme islemi yaptik

In [17]:
y_pred = pcr_model.predict(x_reduced_test)

In [18]:
np.sqrt(mean_squared_error(y_test, y_pred))

# test seti icin hata hesaplama

405.1575364149963